In [1]:
%run ../utils.ipynb

[2024-01-15 14:07:12,613 INFO] [numexpr.utils.utils._init_num_threads] Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [2]:
import os
import sys

from dotenv import load_dotenv, find_dotenv

from yfpy import Data
from yfpy.logger import get_logger
from yfpy.query import YahooFantasySportsQuery

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json

[2024-01-15 14:07:30,506 DEBUG] [matplotlib.__init__.wrapper] matplotlib data path: C:\Users\mikem\anaconda3\Lib\site-packages\matplotlib\mpl-data
[2024-01-15 14:07:30,510 DEBUG] [matplotlib.__init__.wrapper] CONFIGDIR=C:\Users\mikem\.matplotlib
[2024-01-15 14:07:30,512 DEBUG] [matplotlib.__init__.<module>] interactive is False
[2024-01-15 14:07:30,514 DEBUG] [matplotlib.__init__.<module>] platform is win32
[2024-01-15 14:07:30,578 DEBUG] [matplotlib.__init__.wrapper] CACHEDIR=C:\Users\mikem\.matplotlib
[2024-01-15 14:07:30,582 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from C:\Users\mikem\.matplotlib\fontlist-v330.json
[2024-01-15 14:07:30,582 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from C:\Users\mikem\.matplotlib\fontlist-v330.json


In [3]:
auth_dir = f"{project_dir}/auth"

In [4]:
Game_Keys

,season,code
16,2015,348
17,2016,359
18,2017,371
19,2018,380
20,2019,390
21,2020,399
22,2021,406
23,2022,414
24,2023,423


In [5]:
League_Ids

{'2015': 853925,
 '2016': 810238,
 '2017': 192551,
 '2018': 799346,
 '2019': 820083,
 '2020': 394682,
 '2021': 364325,
 '2022': 564704,
 '2023': 333146}

In [6]:
yahoo_query = YahooFantasySportsQuery(
    auth_dir,
    league_id= 333146,
    game_code='nfl',
    game_id= 423
)

In [27]:
teams = yahoo_query.get_league_teams()
team_ids_list = []
for team in teams:
    team_name = str(team.name)[1:].strip("'").strip('"')
    manager = team.managers[0].nickname
    id = team.team_id
    info = {'team_name': team_name, 'manager_nickname': manager, 'id': id}
    team_ids_list.append(info)

#stash team data in dataframe
team_ids_df = pd.DataFrame(team_ids_list)

In [30]:
team_ids_df

,team_name,manager_nickname,id
0,The Ice Cold Bruschi,Ricky,1
1,Immortal Duffmen,Pat,2
2,Jadeveon's Hit Men,Tim,3
3,John Brown's Army,Eric,4
4,Saquon These Nuts,Frags,5
5,Starr is Born,Nick,6
6,SuperCam That Ho,Lauren,7
7,The fishingtons,Buddy,8
8,Gibbs Me A Break,Kyle,9
9,The Twelve Apostles,Jones,10


In [31]:
Query_Auth_Ids

{'2015': [853925, 348],
 '2016': [810238, 359],
 '2017': [192551, 371],
 '2018': [799346, 380],
 '2019': [820083, 390],
 '2020': [394682, 399],
 '2021': [364325, 406],
 '2022': [564704, 414],
 '2023': [333146, 423]}

In [33]:
df = pd.DataFrame()
for year in Query_Auth_Ids.keys():
    yahoo_query = YahooFantasySportsQuery(
    auth_dir,
    league_id= Query_Auth_Ids[year][0],
    game_code='nfl',
    game_id= Query_Auth_Ids[year][1]
    )
    print(f"Query made for season {year}")
    teams = yahoo_query.get_league_teams()
    team_ids_list = []
    for team in teams:
        team_name = str(team.name)[1:].strip("'").strip('"')
        manager = team.managers[0].nickname
        id = team.team_id
        info = {'team_name': team_name, 'manager_nickname': manager, 'id': id}
        team_ids_list.append(info)
    
    #stash team data in dataframe
    team_ids_df = pd.DataFrame(team_ids_list)
    team_ids_df['manager_nickname'] = team_ids_df['team_name'].apply(map_teams_to_name)
    print("Generating player data for:")
    for name, id in zip(team_ids_df['manager_nickname'], team_ids_df['id']):
        try:
            roster = yahoo_query.get_team_roster_player_stats(id)
        except:
            print(f"Error encountered generating roster for manager: {name} in season {year}")
            break
        else:
            print(f"{name}", end="|")
            for player in roster:
                row = pd.json_normalize(player.serialized())
                row['season'] = year
                row['ownership'] = name
                df = pd.concat([df, row])

Query made for season 2015
Generating player data for:
Ricky|Peter|Tim|Big Rick|Michael|Lauren|Jake|Eric|Kyle|Buddy|Pat|Nick|Frags|Jones|Query made for season 2016
Generating player data for:
Ricky|Pat|Tim|Eric|Jones|Lauren|Nick|Buddy|Kyle|Michael|Jake|Frags|Peter|Nate|Query made for season 2017
Generating player data for:
Ricky|Pat|Tim|Eric|Jones|Lauren|Nick|Buddy|Kyle|Michael|Jake|Frags|Peter|Nate|Query made for season 2018
Generating player data for:
Ricky|Pat|Tim|Eric|Jones|Lauren|Nick|Buddy|Kyle|Michael|Jake|Frags|Peter|Nate|Query made for season 2019
Generating player data for:
Ricky|Pat|Tim|Eric|Frags|Nick|Lauren|Buddy|Kyle|Jones|Michael|Jake|Nate|Peter|Query made for season 2020
Generating player data for:
Ricky|Pat|Tim|Eric|Frags|Nick|Lauren|Buddy|Kyle|Jones|Michael|Jake|Nate|Peter|Query made for season 2021
Generating player data for:
Ricky|Pat|Tim|Eric|Frags|Nick|Lauren|Buddy|Kyle|Jones|Michael|Jake|Nate|Peter|Query made for season 2022
Generating player data for:
Ricky|Pat|

In [36]:
df.to_csv(project_dir + 'data\\rosters\\season_rosters_raw.csv')

In [24]:
df = pd.DataFrame()

In [25]:
for player in test:
    row = pd.json_normalize(player.serialized())
    row['season'] = year
    row['ownership'] = 
    df = pd.concat([df, ])

In [26]:
df

,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,has_player_notes,has_recent_player_notes,image_url,...,player_stats.coverage_type,player_stats.season,player_stats.stats,selected_position.coverage_type,selected_position.is_flex,selected_position.position,selected_position.week,injury_note,status,status_full
0,QB,nfl.p.29235,Det,Detroit Lions,nfl.t.8,https://sports.yahoo.com/nfl/teams/detroit/,[QB],1.0,1.0,https://s.yimg.com/iu/api/res/1.2/PPZidVKgBCXz...,...,season,2023,"[{'stat': Stat({ ""stat_id"": 4, ""value"": 45...",week,0,QB,17,NaN,NaN,NaN
0,WR,nfl.p.32695,SF,San Francisco 49ers,nfl.t.25,https://sports.yahoo.com/nfl/teams/san-francisco/,[WR],1.0,NaN,https://s.yimg.com/iu/api/res/1.2/o4KGP6l_rQt9...,...,season,2023,"[{'stat': Stat({ ""stat_id"": 4, ""value"": 0....",week,0,WR,17,NaN,NaN,NaN
0,WR,nfl.p.28534,Buf,Buffalo Bills,nfl.t.2,https://sports.yahoo.com/nfl/teams/buffalo/,[WR],1.0,NaN,https://s.yimg.com/iu/api/res/1.2/80tJkiyYc0vr...,...,season,2023,"[{'stat': Stat({ ""stat_id"": 4, ""value"": 0....",week,0,WR,17,NaN,NaN,NaN
0,RB,nfl.p.34120,LAR,Los Angeles Rams,nfl.t.14,https://sports.yahoo.com/nfl/teams/la-rams/,[RB],1.0,1.0,https://s.yimg.com/iu/api/res/1.2/Lj0ShQWk.ilr...,...,season,2023,"[{'stat': Stat({ ""stat_id"": 4, ""value"": 0....",week,0,RB,17,Hand,Q,Questionable
0,RB,nfl.p.34078,LV,Las Vegas Raiders,nfl.t.13,https://sports.yahoo.com/nfl/teams/las-vegas/,[RB],1.0,NaN,https://s.yimg.com/iu/api/res/1.2/WXLUFGd5_9yE...,...,season,2023,"[{'stat': Stat({ ""stat_id"": 4, ""value"": 0....",week,0,RB,17,NaN,NaN,NaN
0,TE,nfl.p.34085,Dal,Dallas Cowboys,nfl.t.6,https://sports.yahoo.com/nfl/teams/dallas/,[TE],1.0,1.0,https://s.yimg.com/iu/api/res/1.2/0lyCXGNYBem4...,...,season,2023,"[{'stat': Stat({ ""stat_id"": 4, ""value"": 0....",week,0,TE,17,NaN,NaN,NaN
0,WR,nfl.p.40168,LAR,Los Angeles Rams,nfl.t.14,https://sports.yahoo.com/nfl/teams/la-rams/,[WR],1.0,1.0,https://s.yimg.com/iu/api/res/1.2/OhhzcGfxkOl1...,...,season,2023,"[{'stat': Stat({ ""stat_id"": 4, ""value"": 0....",week,1,W/R/T,17,NaN,NaN,NaN
0,QB,nfl.p.5228,TB,Tampa Bay Buccaneers,nfl.t.27,https://sports.yahoo.com/nfl/teams/tampa-bay/,[QB],NaN,NaN,https://s.yimg.com/iu/api/res/1.2/Ol_o6nNm9irL...,...,season,2023,"[{'stat': Stat({ ""stat_id"": 4, ""value"": 0....",week,0,BN,17,NaN,NA,Inactive: Coach's Decision or Not on Roster
0,RB,nfl.p.33996,Sea,Seattle Seahawks,nfl.t.26,https://sports.yahoo.com/nfl/teams/seattle/,[RB],1.0,NaN,https://s.yimg.com/iu/api/res/1.2/P3hgQ_501q_F...,...,season,2023,"[{'stat': Stat({ ""stat_id"": 4, ""value"": 0....",week,0,BN,17,NaN,NaN,NaN
0,TE,nfl.p.31019,Phi,Philadelphia Eagles,nfl.t.21,https://sports.yahoo.com/nfl/teams/philadelphia/,[TE],1.0,NaN,https://s.yimg.com/iu/api/res/1.2/2EigEb4xahJn...,...,season,2023,"[{'stat': Stat({ ""stat_id"": 4, ""value"": 0....",week,0,BN,17,NaN,NaN,NaN
